In [1]:
import os,sys,math,numpy
from PIL import Image
from urllib.request import urlopen
import matplotlib.pyplot as plt

# you must have openvisus installed
# python -m pip install --upgrade OpenVisus
from OpenVisus import *

# LoadImage (example: img=LoadImage("https://github.com/sci-visus/OpenVisus/raw/master/datasets/cat/gray.png"))
def LoadImage(filename):
	return numpy.asarray(PIL.Image.open(urlopen(filename)  if filename.startswith('http') else filename))


Starting OpenVisus c:\python38\lib\site-packages\OpenVisus\__init__.py 3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)] sys.version_info(major=3, minor=8, micro=6, releaselevel='final', serial=0) ...


In [2]:
db=LoadDataset(r'D:\GoogleSci\visus_dataset\TALASS\ACAM\battery.idx')
logic_box=db.getLogicBox()
dims=logic_box[1]
ranges=[(logic_box[0][I],logic_box[1][I]) for I in range(3)]
print("logic_box",logic_box)
print("ranges",ranges)
print("dims",dims)

fields=[db.getField(name) for name in db.getFields()]
for field in fields:
    dtype=field.dtype
    GB=int(dims[0]*dims[1]*dims[2]*dtype.getByteSize()/(1024*1024*1024))
    print("Field",field.name, dtype.toString(), "{}GB".format(GB))

logic_box ([0, 0, 0], [2160, 2160, 2560])
ranges [(0, 2160), (0, 2160), (0, 2560)]
dims [2160, 2160, 2560]
Field xray float32 44GB
Field neutron float32 44GB


In [11]:
def ShowImage(data,width=10):
    if len(data.shape)==3 and data.shape[0]==1: data=data[0,:,:]
    if len(data.shape)==3 and data.shape[1]==1: data=data[:,0,:]   
    if len(data.shape)==3 and data.shape[2]==1: data=data[:,:,0]    
    w,h=data.shape
    my_dpi=96 # monitor dpi
    fig = plt.figure(figsize=(w/my_dpi, h/my_dpi), dpi=my_dpi)
    ax = fig.add_subplot(1,1,1)
    ax.imshow(data, origin='lower', interpolation='None')
    plt.show()

def ShowSlice(axis,max_pixels=1024*1024):

    def ExtractSlice(offset,num_refinements): 
        
        x,y,z=(
            ranges[0] if axis!=0 else [offset,offset+1],
            ranges[1] if axis!=1 else [offset,offset+1],
            ranges[2] if axis!=2 else [offset,offset+1]
        )    

        # guess quality
        quality=0
        tot_pixels=(x[1]-x[0]) * (y[1]-y[0]) * (z[1]-z[0])
        while tot_pixels >= max_pixels:
            quality-=1
            tot_pixels=int(tot_pixels/2)       

        print("Extracting slice","quality",quality,"num_refinements",num_refinements,"x",x,"y",y,"z",z)   
        
        while True:
            try:
                for data in db.read(x=x,y=y,z=z,num_refinements=num_refinements,quality=quality):
                    print("...got data",data.shape,data.dtype) 
                    ShowImage(data)  
                break
            except:
                # it can fail because the quality is too low
                print("increasing quality...")
                quality+=1
    
    
    # c:\Python38\python.exe -m pip install ipywidgets
    # c:\Python38\Scripts\jupyter nbextension enable --py widgetsnbextension
    import ipywidgets
    
    ipywidgets.interact(
        ExtractSlice, 
        offset=ipywidgets.widgets.IntSlider(int(dims[axis]/2),min=0,max=dims[axis]-1,step=1), 
        num_refinements=ipywidgets.widgets.IntSlider(4,min=1,max=6,step=1))


In [13]:
ShowSlice(0)

interactive(children=(IntSlider(value=1080, description='offset', max=2159), IntSlider(value=4, description='n…

In [14]:
ShowSlice(1)

interactive(children=(IntSlider(value=1080, description='offset', max=2159), IntSlider(value=4, description='n…

In [16]:
ShowSlice(2)

interactive(children=(IntSlider(value=1280, description='offset', max=2559), IntSlider(value=4, description='n…